In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 280 kB 5.9 MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.32-cp37-cp37m-linux_x86_64.whl size=287375 sha256=7366409550990f3e1ba9bc46f257d077694cf67198de7e7729ae7a9bf4a7c15e
  Stored in directory: /root/.cache/pip/wheels/2e/9c/da/8652fd42e0f662015554f00a9e96fe4f438dfd1ef59787879e
Successfully built pyodbc


In [ ]:
!pip install eurostat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15147 sha256=83fe0d74edfaf18a21453598b8cde3f995b71c8a4e8224cec6e7c5d1e53fecf7
  Stored in directory: /root/.cache/pip/wheels/58/88/2a/8d619cf38d7cf939e54b6ccdece05d31b64b3eb419c11d1ed3
Successfully built jsonpath-rw


In [ ]:
!apt-get install virtuoso-opensource

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libvirtodbc0 virtuoso-opensource-6.1 virtuoso-opensource-6.1-bin
  virtuoso-opensource-6.1-common virtuoso-server virtuoso-vad-conductor
  virtuoso-vsp-startpage
Suggested packages:
  virtuoso-vad-doc virtuoso-vad-demo virtuoso-vad-tutorial
  virtuoso-vad-rdfmappers virtuoso-vad-sparqldemo virtuoso-vad-syncml
  virtuoso-vad-bpel virtuoso-vad-isparql virtuoso-vad-ods virtuoso-vad-dbpedia
  virtuoso-vad-facetedbrowser
The following NEW packages will be installed:
  libvirtodbc0 virtuoso-opensource virtuoso-opensource-6.1
  virtuoso-opensource-6.1-bin virtuoso-opensource-6.1-common virtuoso-server
  virtuoso-vad-conductor virtuoso-vsp-startpage
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import gensim
import pandas as pd
import numpy as np
import re
import pyodbc

c = pyodbc.connect('DRIVER=/usr/lib/odbc/virtodbc.so;HOST=lod.csd.auth.gr:1111;UID=kimon;PWD=RkhvQYZ442e2JVXLHdtW;DATABASE=ESTAT')
#set encoding
c.setdecoding(pyodbc.SQL_CHAR, encoding='latin-1')
c.setencoding(encoding="latin-1")
cursor = c.cursor()


def load_table(cursor,query):
  cursor.execute(query)
  t1 = cursor.fetchall()
  df = pd.DataFrame.from_records(t1, columns=[x[0] for x in cursor.description])
  return df

#Glossary
query =      """SELECT id,order_id,code_id,term,ramon_cat_id,concept_type_id,definition,context,remark FROM ESTAT.V1.dat_estat_glossary"""

EstatGlossary = load_table(cursor,query)

In [ ]:
#Glossary CODEDTerm mapping
f = open("glossaryLink.ttl", "a")
for name, row in EstatGlossary.iterrows():
  helperGlossary = ""
  id, term, definition, context, remark = row[0], row[3].replace("\"", "").replace("\'", ""), row[6].replace("\"", "").replace("\'", ""), row[7].replace("\"", "").replace("\'", ""), row[8].replace("\"", "").replace("\'", "")
  f.write("estatdata:glossaryterm" + str(id) + " rdf:type estat:CODEDTerm.\n") 
  definition = str(re.sub("[^a-zA-Z\d%]", ' ', definition).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  if term != "":
    term = str(re.sub("[^a-zA-Z\d%]", ' ', term).replace("\'", "").replace("\"", "").replace("\\", "").strip())
    f.write("estatdata:glossaryterm" + str(id) + " estat:term \"" + term + "\"^^xsd:string.\n")
  if definition != "":
    f.write("estatdata:glossaryterm" + str(id) + " estat:definition \"" + definition + "\"^^xsd:string.\n")
  context = str(re.sub("[^a-zA-Z\d%]", ' ', context).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  if  context != "":
    f.write("estatdata:glossaryterm" + str(id) + " estat:context \"" + context + "\"^^xsd:string.\n")
  remerk = str(re.sub("[^a-zA-Z\d%]", ' ', remark).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  if remark != "":
    f.write("estatdata:glossaryTerm" + str(id) + " estat:remark \"" + remark + "\"^^xsd:string.\n")
f.close()

In [ ]:
#Link and shared link
query =      """SELECT id,title,url,resource_information_id,resource_type_id FROM ESTAT.V1.dat_link_info"""

linkInfo = load_table(cursor,query)

query =      """SELECT id,article_id,link_id,article_division_id FROM ESTAT.V1.dat_article_shared_link"""

articleSharedLink = load_table(cursor,query)

In [ ]:
query = """select "ESTAT"."V1"."dat_article"."id", "ESTAT"."V1"."dat_article"."context", "ESTAT"."V1"."dat_article"."data_sources", "ESTAT"."V1"."dat_article"."background_article", "ESTAT"."V1"."dat_article"."homepage", "ESTAT"."V1"."dat_link_info"."id", "ESTAT"."V1"."dat_link_info"."title", "ESTAT"."V1"."dat_link_info"."url", "ESTAT"."V1"."dat_link_info"."resource_information_id", "ESTAT"."V1"."dat_link_info"."resource_type_id" from ( "ESTAT"."V1"."dat_article" left outer join "ESTAT"."V1"."dat_link_info" on "ESTAT"."V1"."dat_article"."id" = "ESTAT"."V1"."dat_link_info"."id" )"""
explainedArticleTitles = load_table(cursor,query)

query = """select "ESTAT"."V1"."dat_glossary"."id", "ESTAT"."V1"."dat_glossary"."definition", "ESTAT"."V1"."dat_glossary"."redirection", "ESTAT"."V1"."dat_glossary"."original_title", "ESTAT"."V1"."dat_glossary"."homepage", "ESTAT"."V1"."dat_glossary"."editorial_note", "ESTAT"."V1"."dat_glossary"."change_note", "ESTAT"."V1"."dat_glossary"."scope_note", "ESTAT"."V1"."dat_glossary"."example", "ESTAT"."V1"."dat_glossary"."history_note", "ESTAT"."V1"."dat_link_info"."id", "ESTAT"."V1"."dat_link_info"."title", "ESTAT"."V1"."dat_link_info"."url", "ESTAT"."V1"."dat_link_info"."resource_information_id", "ESTAT"."V1"."dat_link_info"."resource_type_id" from ( "ESTAT"."V1"."dat_glossary" left outer join "ESTAT"."V1"."dat_link_info" on "ESTAT"."V1"."dat_glossary"."id" = "ESTAT"."V1"."dat_link_info"."id" )"""
glossaryArticlesTitle = load_table(cursor,query)

In [ ]:
listIDsSE = []
for name, row in explainedArticleTitles.iterrows():
  listIDsSE.append((row[5],row[7]))

listIDsGL = []
for name, row in glossaryArticlesTitle.iterrows():
  listIDsGL.append((row[0],row[12]))

In [ ]:
##statistical explained reference 
listLinks = []
for name, row in linkInfo.iterrows():
  title = str(re.sub("[^a-zA-Z\d%]", ' ', row[1]).replace("\'", "").replace("\"", "").replace("\\", "").strip())
  listLinks.append((title, row[2], row[3], row[0]))


f = open("glossaryLink2.ttl", "a")
for name, row in articleSharedLink.iterrows():
  for helper in listLinks:
    if row[2] == helper[3]:
      if listLinks[int(row[2] - 1)][0] != "":
        
        if int(row[3]) ==  0:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estatdata:SEAReference.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  1:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Excel.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  2:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:OtherArticle.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":                
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  3:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Table.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  4:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Database.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  5:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:DedicatedSection.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  6:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Publication.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  7:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Methodology.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  8:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Legislation.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  9:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:Visualization.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

        elif int(row[3]) ==  10:
          f.write("estatdata:reference" + str(row[0]) + " rdf:type estat:ExternalLink.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
          f.write("estatdata:reference" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
          f.write("estatdata:SEArticle" + str(row[1]) + " estat:hasReference estatdata:reference" + str(row[0]) + ".\n")
          if str(listLinks[int(row[2] - 1)][2]) == "1":        
            if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
              for url in listIDsGL:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
            else:
              for url in listIDsSE:
                if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
                  f.write("estatdata:reference" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")

f.close()
 

In [ ]:
query =      """SELECT id,glossary_id,link_id FROM ESTAT.V1.dat_further_info"""

furtherInfo = load_table(cursor,query)

query =      """SELECT id,glossary_id,link_id FROM ESTAT.V1.dat_sources"""

sources = load_table(cursor,query)

query =      """SELECT id,glossary_id,link_id FROM ESTAT.V1.dat_related_concepts"""

relatedConcepts = load_table(cursor,query)

query =      """SELECT id,glossary_id,link_id FROM ESTAT.V1.dat_statistical_data"""

statReferences = load_table(cursor,query)

In [ ]:
##further info glossary articles
f = open("glossaryLink2.ttl", "a")
for name, row in furtherInfo.iterrows():
  if listLinks[int(row[2] - 1)][0] != "":
    f.write("estatdata:referenceFurther" + str(row[0]) + " rdf:type estat:FurtherInfo.\n")
    f.write("estatdata:referenceFurther" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
    f.write("estatdata:referenceFurther" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
    f.write("estatdata:glossaryArticle" + str(row[1]) + " estat:hasReference estatdata:referenceFurther" + str(row[0]) + ".\n")      
    if str(listLinks[int(row[2] - 1)][2]) == "1":        
        if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
          for url in listIDsGL:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceFurther" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
        else:
          for url in listIDsSE:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceFurther" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")
f.close()

In [ ]:
##sources glossary articles
f = open("glossaryLink2.ttl", "a")
for name, row in sources.iterrows():
  if listLinks[int(row[2] - 1)][0] != "":
    f.write("estatdata:referenceSource" + str(row[0]) + " rdf:type estat:Source.\n")
    f.write("estatdata:referenceSource" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
    f.write("estatdata:referenceSource" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
    f.write("estatdata:glossaryArticle" + str(row[1]) + " estat:hasReference estatdata:referenceSource" + str(row[0]) + ".\n")      
    if str(listLinks[int(row[2] - 1)][2]) == "1":        
        if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
          for url in listIDsGL:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceSource" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
        else:
          for url in listIDsSE:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceSource" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")
f.close()

In [ ]:
##relatedConcepts glossary articles
f = open("glossaryLink2.ttl", "a")
for name, row in relatedConcepts.iterrows():
  if listLinks[int(row[2] - 1)][0] != "":
    f.write("estatdata:referenceConcepts" + str(row[0]) + " rdf:type estat:RelatedConcept.\n")
    f.write("estatdata:referenceConcepts" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
    f.write("estatdata:referenceConcepts" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
    f.write("estatdata:glossaryArticle" + str(row[1]) + " estat:hasReference estatdata:referenceConcepts" + str(row[0]) + ".\n")      
    if str(listLinks[int(row[2] - 1)][2]) == "1":        
        if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
          for url in listIDsGL:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceConcepts" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
        else:
          for url in listIDsSE:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceConcepts" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")
f.close()

In [ ]:
##statistical data glossary articles
f = open("glossaryLink2.ttl", "a")
for name, row in statReferences.iterrows():
  if listLinks[int(row[2] - 1)][0] != "":
    f.write("estatdata:referenceData" + str(row[0]) + " rdf:type estat:RelatedStatisticalData.\n")
    f.write("estatdata:referenceData" + str(row[0]) + " estat:title \"" + listLinks[int(row[2] - 1)][0]  + "\"^^xsd:string.\n")
    f.write("estatdata:referenceData" + str(row[0]) + " estat:hasURL \"" + listLinks[int(row[2] - 1)][1] + "\"^^xsd:anyURI.\n")
    f.write("estatdata:glossaryArticle" + str(row[1]) + " estat:hasReference estatdata:referenceData" + str(row[0]) + ".\n")      
    if str(listLinks[int(row[2] - 1)][2]) == "1":        
        if "index.php?title=Glossary:" in listLinks[int(row[2] - 1)][1]:
          for url in listIDsGL:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceData" + str(row[0]) + " estat:hasURI estatdata:glossaryArticle" + str(url[0]) + ".\n")
        else:
          for url in listIDsSE:
            if str(url[1]) == str(listLinks[int(row[2] - 1)][1]):
              f.write("estatdata:referenceData" + str(row[0]) + " estat:hasURI estatdata:SEArticle" + str(url[0]) +".\n")
f.close()